In [13]:
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from helper import load_normalized_data_regression
import pandas as pd
X, y = load_normalized_data_regression()
#X, y = load_data()
y = (y-min(y)) / (max(y)-min(y))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
print(X, y)
print(max(y), min(y))

tensor_x = torch.tensor(X.to_numpy())
tensor_y = torch.tensor(y.to_numpy())

class MyDataset(Dataset):

  def __init__(self,x, y):
    self.X=torch.tensor(x.to_numpy(),dtype=torch.float32)
    self.y=torch.tensor(y.to_numpy(),dtype=torch.float32)

  def __len__(self):
    return len(self.y)
  
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

train_dataset = MyDataset(X_train, y_train)
test_dataset = MyDataset(X_test, y_test)

          qPa     pulso      resp
0    0.077115  0.284192  0.419223
1    0.069330  0.733000  0.663855
2    0.936665  0.570369  0.195441
3    0.932465  0.623926  0.173632
4    0.930480  0.256719  0.956455
..        ...       ...       ...
795  0.075935  0.927780  0.400736
796  0.743465  0.990068  0.494227
797  0.739860  0.920339  0.408064
798  0.912695  0.431128  0.861291
799  0.485410  0.777671  0.908814

[800 rows x 3 columns] 0      0.307063
1      0.833476
2      0.258594
3      0.164530
4      0.266492
         ...   
795    0.001239
796    0.318026
797    0.168328
798    0.608509
799    0.396973
Name: grav, Length: 800, dtype: float64
1.0 0.0


In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler

class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(3, 6),
      nn.ReLU(),
      nn.Linear(6, 6),
      nn.ReLU(),
      nn.Linear(6, 1)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

  
if __name__ == '__main__':
  
  # Set fixed random number seed
  torch.manual_seed(42)
  
  # Load Boston dataset
  
  # Prepare Boston dataset
  dataset = train_dataset
  trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
  
  # Initialize the MLP
  mlp = MLP()
  
  # Define the loss function and optimizer
  loss_function = nn.L1Loss()
  optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)
  
  # Run the training loop
  for epoch in range(0, 500): # 5 epochs at maximum
    
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
      inputs, targets = data
      inputs, targets = inputs.float(), targets.float()
      targets = targets.reshape((targets.shape[0], 1))
      
      # Zero the gradients
      optimizer.zero_grad()
      
      # Perform forward pass
      outputs = mlp(inputs)
      
      # Compute loss
      loss = loss_function(outputs, targets)
      
      # Perform backward pass
      loss.backward()
      
      # Perform optimization
      optimizer.step()
      
      # Print statistics
      current_loss += loss.item()
      if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, current_loss / 500))
          current_loss = 0.0

  # Process is complete.
  print('Training process has finished.')

Starting epoch 1
Loss after mini-batch     1: 0.001
Loss after mini-batch    11: 0.005
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.005
Loss after mini-batch    51: 0.004
Starting epoch 2
Loss after mini-batch     1: 0.001
Loss after mini-batch    11: 0.004
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.004
Loss after mini-batch    51: 0.004
Starting epoch 3
Loss after mini-batch     1: 0.001
Loss after mini-batch    11: 0.004
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.005
Loss after mini-batch    41: 0.003
Loss after mini-batch    51: 0.004
Starting epoch 4
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.004
Loss after mini-batch    21: 0.004
Loss after mini-batch    31: 0.004
Loss after mini-batch    41: 0.004
Loss after mini-batch    51: 0.004
Starting epoch 5
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.004
Loss 

In [17]:
import pandas as pd
predict = mlp(torch.tensor(X_test.to_numpy(), dtype=torch.float32))

df_predict = pd.DataFrame()
df_predict['real'] = y_test
df_predict['previsao'] = predict.detach().numpy()
df_predict

,real,previsao
0,0.589903,0.435960
1,0.104815,0.511232
2,0.179782,0.453780
3,0.000726,0.006572
4,0.759755,0.461993
...,...,...
235,0.627952,0.466205
236,0.810129,0.414367
237,0.754075,0.414699
238,0.626225,0.449437
